In [1]:
from langchain_core.prompts import PromptTemplate

from configs.agent import PROMPT_TEMPLATE

prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)

In [2]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from utils.agent import load_reader_model
from tools.happyface_search import HappyFaceSearchResultsTool
from tools.personal_consultancy_search import PersonalConsultancySearchTool
from tools.financial_capability_search import FinancialCapabilitySearchTool
from tools.time_capability_search import TimeCapabilitySearchTool
from tools.current_datetime_search import CurrentDateTimeSearchTool
from utils.web_search import load_web_search_answer_tool

llm = load_reader_model()
WebSearchAnswerTool = load_web_search_answer_tool()
tools = [
    HappyFaceSearchResultsTool(),
    PersonalConsultancySearchTool(),
    FinancialCapabilitySearchTool(),
    TimeCapabilitySearchTool(),
    CurrentDateTimeSearchTool(),
    WebSearchAnswerTool
]
agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [3]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description(tools)
tool_names = ', '.join([tool.name for tool in tools])
print(rendered_tools)
print(tool_names)

happyface_search_results_tool: A search tool optimized for comprehensive, accurate, and trusted results of HappyFace service. Useful when you need to answer questions about features and services about HappyFace platform. Input should be a search query.
personal_consultancy_search_tool: A consultancy search tool optimized for comprehensive, accurate, and trusted consultations on personal issues. Useful when you need to answer questions about personal challenges and emotionally impacting human matters of individuals. Inputs should be a query and a profile recommendation.
financial_capability_search_tool: A search tool optimized for comprehensive, accurate, and trusted results of personal financial capabilities. Useful when you need to answer questions about financial capabilities of a given individual by employee id. This tool supports decision making on available loans, offers and salary deductions for an individual who currently works in the given organization. Input should be an emplo

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []


def get_agent_output(
        query: str,
        organization_name: str,
        employee_id: int | str,
        recommendation: str
) -> str:
    result = agent_executor.invoke({
        "query": query,
        "organization_name": organization_name,
        "employee_id": employee_id,
        "recommendation": recommendation,
        "chat_history": chat_history,
        "tools": rendered_tools,
        "tool_names": tool_names
    })
    return result["output"]


def append_latest_conv_to_chat_history(query: str, output: str) -> None:
    latest_conv = [
        HumanMessage(content=query),
        AIMessage(content=output)
    ]
    chat_history.append(latest_conv)

In [5]:
from utils.profile import generate_summarized_profile, generate_profiles_into_recommendation

organization = "ABC Corporation"
employee = "E1"
bio_data_profile = {
    # "name": "Sineth",
    "address": "Sri Lanka, Colombo",
    "dob": "2000-09-18",
    "gender": "male",
    "salary (LKR)": 10000,
    "hiddenDiseases": [],
    "family": {
        "numMembers": 5,
        "monthlyIncome (LKR)": 30000,
        "monthlyExpenses (LKR)": 28000,
        "category": "nucleus"
    }
}
emotion_engagement_profile = {
    # "name": "Smith",
    # "gender": "male",
    "emotionalProfile": {
        "mostlyEngagingFacialEmotion": {
            "emotion": "happy",
            "avgArousal (percentage)": 90,
            "avgValence (percentage)": 60
        },
        "mostlyEngagingSentimentalEmotion": {
            "emotion": "happy",
            "avgArousal (percentage)": 30,
            "avgValence (percentage)": 20
        }
    }
}
bio_data_summary = generate_summarized_profile(bio_data_profile)
emotion_engagement_summary = generate_summarized_profile(emotion_engagement_profile)
profile_recommendation = generate_profiles_into_recommendation(
    bio_data_summary,
    emotion_engagement_summary
)

In [6]:
print(profile_recommendation)

 Taking into account the provided personal bio data and emotional engagement profiles, several recommendations can be made for this individual.

Firstly, considering the balanced financial situation of his nucleus family, it is suggested that this employee maintains his current employment status. This will ensure the stability of his family's income and expenses. Furthermore, with a salary of 10,000 LKR, he is contributing significantly to the overall family income, which is currently 30,000 LKR per month.

Secondly, the employee's emotional disposition indicates a predominant engagement with the emotion "happy." This is a positive attribute that can be leveraged in the workplace. Managers should consider assigning this employee tasks that require creativity and innovation, as positive emotions have been linked to enhanced problem-solving abilities and increased productivity.

Additionally, the difference in arousal levels between the two "happy" emotions suggests varying intensities o

In [14]:
query = "I am in a severe stress condition a personal financial un stability :("
output = get_agent_output(query, organization, employee, profile_recommendation)
append_latest_conv_to_chat_history(query, output)
print(output)



> Entering new AgentExecutor chain...


APIError: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_d8a88ac45505b00f43561e7991d71411 in your email.)

In [12]:
query = "How can I get organizational support in this challenging financial period?"
output = get_agent_output(query, organization, employee, profile_recommendation)
append_latest_conv_to_chat_history(query, output)
print(output)



> Entering new AgentExecutor chain...


APIError: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_5c64d0a378f25ccb8e8b342769e63eae in your email.)

In [8]:
query = "What are available offers that I can be taken from my organization in this week?"
output = get_agent_output(query, organization, employee, profile_recommendation)
append_latest_conv_to_chat_history(query, output)
print(output)



> Entering new AgentExecutor chain...

Invoking: `financial_capability_search_tool` with `{'employee_id': 'E1'}`


{'employee_id': 'E1', 'availableLoans': [{'id': 1, 'name': 'medical', 'amount': 100000, 'currency': 'LKR', 'periodToSettle': {'years': 1, 'months': 6, 'days': 0}}, {'id': 2, 'name': 'personal', 'amount': 50000, 'currency': 'LKR', 'periodToSettle': {'years': 0, 'months': 6, 'days': 0}}], 'availableOffers': [{'id': 1, 'name': 'christmas', 'amount': 5000, 'currency': 'LKR', 'getActiveOn': {'month': 'December', 'date': 1}}], 'salaryDeductions': [{'id': 1, 'reason': 'company equipment damage', 'amount': 1000, 'currency': 'LKR', 'basis': 'per month'}]}
Invoking: `financial_capability_search_tool` with `{'employee_id': 'E1'}`


{'employee_id': 'E1', 'availableLoans': [{'id': 1, 'name': 'medical', 'amount': 100000, 'currency': 'LKR', 'periodToSettle': {'years': 1, 'months': 6, 'days': 0}}, {'id': 2, 'name': 'personal', 'amount': 50000, 'currency': 'LKR', 'periodToSettle': {'year

APIError: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_5548d88188aa7cf926f894bcc8a83bbe in your email.)

In [10]:
query = "Is tomorrow a holiday for me?"
output = get_agent_output(query, organization, employee, profile_recommendation)
append_latest_conv_to_chat_history(query, output)
print(output)



> Entering new AgentExecutor chain...

Invoking: `time_capability_search_tool` with `{'employee_id': 'E1'}`


{'employee_id': 'E1', 'workingSlots': [{'id': 1, 'day': 'monday', 'from': '8.00 AM', 'to': '6.00 PM'}, {'id': 2, 'day': 'tuesday', 'from': '8.00 AM', 'to': '6.00 PM'}, {'id': 3, 'day': 'wednesday', 'from': '8.00 AM', 'to': '3.00 PM'}, {'id': 4, 'day': 'thursday', 'from': '8.00 AM', 'to': '6.00 PM'}, {'id': 5, 'day': 'friday', 'from': '8.00 AM', 'to': '12.00 AM'}], 'numOfHolidaysAvailable': 12, 'vacations': [{'id': 1, 'name': 'christmas', 'from': {'date': 23, 'month': 'december'}, 'to': {'date': 1, 'month': 'january'}}]}
Invoking: `time_capability_search_tool` with `{'employee_id': 'E1'}`


{'employee_id': 'E1', 'workingSlots': [{'id': 1, 'day': 'monday', 'from': '8.00 AM', 'to': '6.00 PM'}, {'id': 2, 'day': 'tuesday', 'from': '8.00 AM', 'to': '6.00 PM'}, {'id': 3, 'day': 'wednesday', 'from': '8.00 AM', 'to': '3.00 PM'}, {'id': 4, 'day': 'thursday', 'from': '8.00 AM', 'to': '6

APIError: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_4019bcb702d541731a856c031617aaa6 in your email.)

In [11]:
from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the datetime object into a readable string
formatted_date_time = now.strftime("%A, %d %B %Y %I:%M %p")

print("Current datetime:", formatted_date_time)

Current datetime: Tuesday, 09 April 2024 03:00 PM
